In [ ]:
# pip/conda 'magic' commands % in Jupyter
# pip install bs4
# %conda install requests

In [15]:
# colab only

apikey = '' #@param {type:"string"}

In [16]:
import requests
import os
from bs4 import BeautifulSoup

In [ ]:
# for Jupyter / VSCode
# apikey = os.environ['TRANSLINK_KEY']

In [8]:
############# DO NOT DO THIS ################
# store your API key(s) in environment variables
# apikey

**Documentation:** <br>
https://www.translink.ca/about-us/doing-business-with-translink/app-developer-resources/rtti

## Basic Response from API

In [18]:
URL = f'http://api.translink.ca/rttiapi/v1/routes/003?apikey={apikey}'
response = requests.get(URL) # string containing xml

In [22]:
if response.status_code == 200:
  print(response.text)
else:
  print('the call failed', response.status_code)

<Route xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><RouteNo>003</RouteNo><Name>MAIN/WATERFRONT STN</Name><OperatingCompany>CMBC</OperatingCompany><Patterns><Pattern><PatternNo>N1</PatternNo><Destination>MAIN/TO WATERFRONT STN</Destination><RouteMap><Href>https://nb.translink.ca/geodata/trip/003-N1.kmz</Href></RouteMap><Direction>NORTH</Direction></Pattern><Pattern><PatternNo>S1</PatternNo><Destination>MAIN/TO MARINE DR STN</Destination><RouteMap><Href>https://nb.translink.ca/geodata/trip/003-S1.kmz</Href></RouteMap><Direction>SOUTH</Direction></Pattern></Patterns></Route>


In [23]:
print(response.text)

<Route xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><RouteNo>003</RouteNo><Name>MAIN/WATERFRONT STN</Name><OperatingCompany>CMBC</OperatingCompany><Patterns><Pattern><PatternNo>N1</PatternNo><Destination>MAIN/TO WATERFRONT STN</Destination><RouteMap><Href>https://nb.translink.ca/geodata/trip/003-N1.kmz</Href></RouteMap><Direction>NORTH</Direction></Pattern><Pattern><PatternNo>S1</PatternNo><Destination>MAIN/TO MARINE DR STN</Destination><RouteMap><Href>https://nb.translink.ca/geodata/trip/003-S1.kmz</Href></RouteMap><Direction>SOUTH</Direction></Pattern></Patterns></Route>


### Parsing XML

`pip install lxml`


In [24]:
soup = BeautifulSoup(response.text, 'lxml')
soup

<html><body><route xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><routeno>003</routeno><name>MAIN/WATERFRONT STN</name><operatingcompany>CMBC</operatingcompany><patterns><pattern><patternno>N1</patternno><destination>MAIN/TO WATERFRONT STN</destination><routemap><href>https://nb.translink.ca/geodata/trip/003-N1.kmz</href></routemap><direction>NORTH</direction></pattern><pattern><patternno>S1</patternno><destination>MAIN/TO MARINE DR STN</destination><routemap><href>https://nb.translink.ca/geodata/trip/003-S1.kmz</href></routemap><direction>SOUTH</direction></pattern></patterns></route></body></html>

In [ ]:
print(soup.prettify())

<html>
 <body>
  <route xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
   <routeno>
    003
   </routeno>
   <name>
    MAIN/WATERFRONT STN
   </name>
   <operatingcompany>
    CMBC
   </operatingcompany>
   <patterns>
    <pattern>
     <patternno>
      N1
     </patternno>
     <destination>
      MAIN/TO WATERFRONT STN
     </destination>
     <routemap>
      <href>
       https://nb.translink.ca/geodata/trip/003-N1.kmz
      </href>
     </routemap>
     <direction>
      NORTH
     </direction>
    </pattern>
    <pattern>
     <patternno>
      S1
     </patternno>
     <destination>
      MAIN/TO MARINE DR STN
     </destination>
     <routemap>
      <href>
       https://nb.translink.ca/geodata/trip/003-S1.kmz
      </href>
     </routemap>
     <direction>
      SOUTH
     </direction>
    </pattern>
   </patterns>
  </route>
 </body>
</html>



In [25]:
# Example 1: get information about a specific bus stop
URL = f'http://api.translink.ca/rttiapi/v1/stops/61935?apikey={apikey}'

response = requests.get(URL)
soup = BeautifulSoup(response.text, 'lxml')
print(soup.prettify())

<html>
 <body>
  <stop xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
   <stopno>
    61935
   </stopno>
   <name>
    UBC EXCHANGE BAY 7
   </name>
   <bayno>
    4
   </bayno>
   <city>
    VANCOUVER
   </city>
   <onstreet>
    UBC EXCHANGE
   </onstreet>
   <atstreet>
    BAY 7
   </atstreet>
   <latitude>
    49.267419
   </latitude>
   <longitude>
    -123.246831
   </longitude>
   <wheelchairaccess>
    1
   </wheelchairaccess>
   <distance>
    -1
   </distance>
   <routes>
    099
   </routes>
  </stop>
 </body>
</html>



In [26]:
# extract the stop number
latitude = soup.find('latitude').text
latitude

'49.267419'

In [27]:
# extract the city
city = soup.find('city').text
city.capitalize()

'Vancouver'

In [28]:
# extract on street
on_street = soup.find('onstreet')
on_street.text

'UBC EXCHANGE'

### Parsing JSON

In [29]:
# Example 2: get estimates of bus schedules at a specific stop
request_url = f'http://api.translink.ca/rttiapi/v1/stops/61935/estimates?apikey={apikey}'
header_dict = {'accept': 'application/JSON'}
response = requests.get(request_url, headers=header_dict)

In [31]:
response.text

'[{"RouteNo":"099","RouteName":"COMMERCIAL-BROADWAY\\/UBC (B-LINE)","Direction":"EAST","RouteMap":{"Href":"https:\\/\\/nb.translink.ca\\/geodata\\/099.kmz"},"Schedules":[{"Pattern":"E1","Destination":"COMM\'L-BDWAY STN","ExpectedLeaveTime":"6:24pm 2024-03-25","ExpectedCountdown":0,"ScheduleStatus":"-","CancelledTrip":false,"CancelledStop":false,"AddedTrip":false,"AddedStop":false,"LastUpdate":"06:24:27 pm"},{"Pattern":"E8FL2","Destination":"TO BOUNDARY B-LINE","ExpectedLeaveTime":"6:24pm 2024-03-25","ExpectedCountdown":0,"ScheduleStatus":"-","CancelledTrip":false,"CancelledStop":false,"AddedTrip":false,"AddedStop":false,"LastUpdate":"06:24:25 pm"},{"Pattern":"E1","Destination":"COMM\'L-BDWAY STN","ExpectedLeaveTime":"6:28pm 2024-03-25","ExpectedCountdown":4,"ScheduleStatus":"-","CancelledTrip":false,"CancelledStop":false,"AddedTrip":false,"AddedStop":false,"LastUpdate":"06:11:31 pm"},{"Pattern":"E1","Destination":"COMM\'L-BDWAY STN","ExpectedLeaveTime":"6:33pm 2024-03-25","ExpectedCoun

In [33]:
d = response.json()
type(d[0])

dict

In [38]:
res_json = response.json()
res_json[0]

{'RouteNo': '099',
 'RouteName': 'COMMERCIAL-BROADWAY/UBC (B-LINE)',
 'Direction': 'EAST',
 'RouteMap': {'Href': 'https://nb.translink.ca/geodata/099.kmz'},
 'Schedules': [{'Pattern': 'E1',
   'Destination': "COMM'L-BDWAY STN",
   'ExpectedLeaveTime': '6:24pm 2024-03-25',
   'ExpectedCountdown': 0,
   'ScheduleStatus': '-',
   'CancelledTrip': False,
   'CancelledStop': False,
   'AddedTrip': False,
   'AddedStop': False,
   'LastUpdate': '06:24:27 pm'},
  {'Pattern': 'E8FL2',
   'Destination': 'TO BOUNDARY B-LINE',
   'ExpectedLeaveTime': '6:24pm 2024-03-25',
   'ExpectedCountdown': 0,
   'ScheduleStatus': '-',
   'CancelledTrip': False,
   'CancelledStop': False,
   'AddedTrip': False,
   'AddedStop': False,
   'LastUpdate': '06:24:25 pm'},
  {'Pattern': 'E1',
   'Destination': "COMM'L-BDWAY STN",
   'ExpectedLeaveTime': '6:28pm 2024-03-25',
   'ExpectedCountdown': 4,
   'ScheduleStatus': '-',
   'CancelledTrip': False,
   'CancelledStop': False,
   'AddedTrip': False,
   'AddedStop'

In [39]:
res_json[0].keys()

dict_keys(['RouteNo', 'RouteName', 'Direction', 'RouteMap', 'Schedules'])

In [ ]:
res_json[0]['Schedules'][0].keys()

dict_keys(['Pattern', 'Destination', 'ExpectedLeaveTime', 'ExpectedCountdown', 'ScheduleStatus', 'CancelledTrip', 'CancelledStop', 'AddedTrip', 'AddedStop', 'LastUpdate'])

In [40]:
for schedule in res_json[0]['Schedules']:
    print('Leaving at ' + schedule['ExpectedLeaveTime'])

Leaving at 6:24pm 2024-03-25
Leaving at 6:24pm 2024-03-25
Leaving at 6:28pm 2024-03-25
Leaving at 6:33pm 2024-03-25
Leaving at 6:40pm 2024-03-25
Leaving at 6:43pm 2024-03-25


In [51]:
# Example 3: get all bus stops near a (latitude, longitude) coordinate (more parameters)
request_url = f'https://api.translink.ca/rttiapi/v1/stops?apikey={apikey}&lat=49.18&long=-122.85&radius=600'
response = requests.get(request_url, headers={'accept': 'application/JSON'})
response.json()

[{'StopNo': 54997,
  'Name': 'SB KING GEORGE BLVD FS 98 AVE',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': 'KING GEORGE BLVD',
  'AtStreet': '98 AVE',
  'Latitude': 49.179601,
  'Longitude': -122.845814,
  'WheelchairAccess': 1,
  'Distance': 307,
  'Routes': '314, 321, 329'},
 {'StopNo': 54986,
  'Name': 'EB 96 AVE FS 134 ST',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': '96 AVE',
  'AtStreet': '134 ST',
  'Latitude': 49.176992,
  'Longitude': -122.850709,
  'WheelchairAccess': 2,
  'Distance': 338,
  'Routes': '314, 329'},
 {'StopNo': 54999,
  'Name': 'WB 96 AVE FS 134 ST',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': '96 AVE',
  'AtStreet': '134 ST',
  'Latitude': 49.177159,
  'Longitude': -122.851794,
  'WheelchairAccess': 2,
  'Distance': 342,
  'Routes': '314, 329'},
 {'StopNo': 54989,
  'Name': 'NB KING GEORGE BLVD FS FRASER HWY',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': 'KING GEORGE BLVD',
  'AtStreet': 'FRASER HWY',
  'Latitude': 49.181116,
  'Longitude

In [52]:
len(response.json())

21

## Using the requests.get() 'params' parameter

In [53]:
URL = 'https://api.translink.ca/rttiapi/v1/stops'
params_dict = {
    'lat' : '49.18',
    'long' : '-122.85',
    'radius' : '600',
    'apikey' : apikey
}
headers_dict = {'accept': 'application/JSON'}
response = requests.get(URL, params=params_dict, headers=headers_dict)
res_json = response.json()
res_json

[{'StopNo': 54997,
  'Name': 'SB KING GEORGE BLVD FS 98 AVE',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': 'KING GEORGE BLVD',
  'AtStreet': '98 AVE',
  'Latitude': 49.179601,
  'Longitude': -122.845814,
  'WheelchairAccess': 1,
  'Distance': 307,
  'Routes': '314, 321, 329'},
 {'StopNo': 54986,
  'Name': 'EB 96 AVE FS 134 ST',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': '96 AVE',
  'AtStreet': '134 ST',
  'Latitude': 49.176992,
  'Longitude': -122.850709,
  'WheelchairAccess': 2,
  'Distance': 338,
  'Routes': '314, 329'},
 {'StopNo': 54999,
  'Name': 'WB 96 AVE FS 134 ST',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': '96 AVE',
  'AtStreet': '134 ST',
  'Latitude': 49.177159,
  'Longitude': -122.851794,
  'WheelchairAccess': 2,
  'Distance': 342,
  'Routes': '314, 329'},
 {'StopNo': 54989,
  'Name': 'NB KING GEORGE BLVD FS FRASER HWY',
  'BayNo': 'N',
  'City': 'SURREY',
  'OnStreet': 'KING GEORGE BLVD',
  'AtStreet': 'FRASER HWY',
  'Latitude': 49.181116,
  'Longitude

In [54]:
for stop in res_json:
    print(stop['WheelchairAccess'])

1
2
2
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1


In [55]:
wheelchair_access = {}
for stop in res_json:
    access = stop['WheelchairAccess']
    name = stop['Name']
    if access == 1:
        wheelchair_access[name] = True
    else:
        wheelchair_access[name] = False

wheelchair_access

{'SB KING GEORGE BLVD FS 98 AVE': True,
 'EB 96 AVE FS 134 ST': False,
 'WB 96 AVE FS 134 ST': False,
 'NB KING GEORGE BLVD FS FRASER HWY': True,
 'WB 96 AVE FS KING GEORGE BLVD': True,
 'KING GEORGE STN BAY 1': True,
 'NB KING GEORGE BLVD FS 96 AVE': True,
 'KING GEORGE STN BAY 4': True,
 'KING GEORGE STATION PLATFORM 2': True,
 'NB 132 ST NS 98 AVE': True,
 'SB 132 ST FS 98 AVE': False,
 'KING GEORGE STN BAY 2': True,
 'KING GEORGE STATION PLATFORM 1': True,
 'EB OLD YALE RD AT 13500 BLOCK': True,
 'KING GEORGE STN BAY 5': True,
 'SB KING GEORGE BLVD FS 96 AVE': True,
 'EB 96 AVE FS 132 ST': True,
 'NB 132 ST FS 96 AVE': True,
 'WB FRASER HWY FS WHALLEY BLVD': True}

## Now with XML

In [56]:
URL = 'https://api.translink.ca/rttiapi/v1/stops'

params_dict = {
    'lat' : '49.18',
    'long' : '-122.85',
    'radius' : '600',
    'apikey' : apikey
}
response = requests.get(URL, params=params_dict)
response.text

'<Stops xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><Stop><StopNo>54997</StopNo><Name>SB KING GEORGE BLVD FS 98 AVE</Name><BayNo>N</BayNo><City>SURREY</City><OnStreet>KING GEORGE BLVD</OnStreet><AtStreet>98 AVE</AtStreet><Latitude>49.179601</Latitude><Longitude>-122.845814</Longitude><WheelchairAccess>1</WheelchairAccess><Distance>307</Distance><Routes>314, 321, 329</Routes></Stop><Stop><StopNo>54986</StopNo><Name>EB 96 AVE FS 134 ST</Name><BayNo>N</BayNo><City>SURREY</City><OnStreet>96 AVE</OnStreet><AtStreet>134 ST</AtStreet><Latitude>49.176992</Latitude><Longitude>-122.850709</Longitude><WheelchairAccess>2</WheelchairAccess><Distance>338</Distance><Routes>314, 329</Routes></Stop><Stop><StopNo>54999</StopNo><Name>WB 96 AVE FS 134 ST</Name><BayNo>N</BayNo><City>SURREY</City><OnStreet>96 AVE</OnStreet><AtStreet>134 ST</AtStreet><Latitude>49.177159</Latitude><Longitude>-122.851794</Longitude><WheelchairAccess>2</WheelchairAccess><Distance>342</Distance><Routes>314, 329</Routes><

In [57]:
soup = BeautifulSoup(response.text, 'lxml')
print(soup.prettify())

<html>
 <body>
  <stops xmlns:i="http://www.w3.org/2001/XMLSchema-instance">
   <stop>
    <stopno>
     54997
    </stopno>
    <name>
     SB KING GEORGE BLVD FS 98 AVE
    </name>
    <bayno>
     N
    </bayno>
    <city>
     SURREY
    </city>
    <onstreet>
     KING GEORGE BLVD
    </onstreet>
    <atstreet>
     98 AVE
    </atstreet>
    <latitude>
     49.179601
    </latitude>
    <longitude>
     -122.845814
    </longitude>
    <wheelchairaccess>
     1
    </wheelchairaccess>
    <distance>
     307
    </distance>
    <routes>
     314, 321, 329
    </routes>
   </stop>
   <stop>
    <stopno>
     54986
    </stopno>
    <name>
     EB 96 AVE FS 134 ST
    </name>
    <bayno>
     N
    </bayno>
    <city>
     SURREY
    </city>
    <onstreet>
     96 AVE
    </onstreet>
    <atstreet>
     134 ST
    </atstreet>
    <latitude>
     49.176992
    </latitude>
    <longitude>
     -122.850709
    </longitude>
    <wheelchairaccess>
     2
    </wheelchairaccess>
    <dis

In [58]:
stops = soup.find_all('stop')
print(stops[0].prettify())

<stop>
 <stopno>
  54997
 </stopno>
 <name>
  SB KING GEORGE BLVD FS 98 AVE
 </name>
 <bayno>
  N
 </bayno>
 <city>
  SURREY
 </city>
 <onstreet>
  KING GEORGE BLVD
 </onstreet>
 <atstreet>
  98 AVE
 </atstreet>
 <latitude>
  49.179601
 </latitude>
 <longitude>
  -122.845814
 </longitude>
 <wheelchairaccess>
  1
 </wheelchairaccess>
 <distance>
  307
 </distance>
 <routes>
  314, 321, 329
 </routes>
</stop>



In [59]:
for stop in stops:
    print(stop.find('wheelchairaccess').text)

1
2
2
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1


In [60]:
wheelchair_access = {}
for stop in stops:
    access = stop.find('wheelchairaccess').text
    # use element.text attribute
    name = stop.find('name').text
    if access == '1':
        wheelchair_access[name] = True
    else:
        wheelchair_access[name] = False

wheelchair_access

{'SB KING GEORGE BLVD FS 98 AVE': True,
 'EB 96 AVE FS 134 ST': False,
 'WB 96 AVE FS 134 ST': False,
 'NB KING GEORGE BLVD FS FRASER HWY': True,
 'WB 96 AVE FS KING GEORGE BLVD': True,
 'KING GEORGE STN BAY 1': True,
 'NB KING GEORGE BLVD FS 96 AVE': True,
 'KING GEORGE STN BAY 4': True,
 'KING GEORGE STATION PLATFORM 2': True,
 'NB 132 ST NS 98 AVE': True,
 'SB 132 ST FS 98 AVE': False,
 'KING GEORGE STN BAY 2': True,
 'KING GEORGE STATION PLATFORM 1': True,
 'EB OLD YALE RD AT 13500 BLOCK': True,
 'KING GEORGE STN BAY 5': True,
 'SB KING GEORGE BLVD FS 96 AVE': True,
 'EB 96 AVE FS 132 ST': True,
 'NB 132 ST FS 96 AVE': True,
 'WB FRASER HWY FS WHALLEY BLVD': True}